#### I have not had any collaborators.

In [1]:
import json_lines
import numpy 
import os 
import re 
import binascii 
from time import time
from os import listdir
import itertools
import tarfile
import nltk
from collections import Counter
import pandas as pd


__1. Read tweets in python__

__2. Extract the actual text fields from the tweet jsons, discard all metadata at this point.               Note that sometimes the text may be truncated to fit the old character limit. In these                cases, is it possible to get the full text?__
    Yes, this has been done.

In [2]:
def get_data():
    '''
    Extract tweets (truncated and nontruncated), 
    lowercase everything, 
    remove link to tweets at the end 
    '''
    data = []
    with open('english-tw
              eets-sample.jsonl', 'rb') as f:
        for item in json_lines.reader(f):            
            # if truncated, you can access the extended tweets
            if item['truncated']==True:
                data.append(item['extended_tweet']['full_text'].lower().split('https:')[0]) 
            else:
                data.append(item['text'].lower().split('https:')[0])
    return data

In [3]:
data = get_data()

__3. Segment each tweet using both UDPipe machine learned model (can be found from             the same data package) and a heuristic method. What can you tell about the              segmentation performance on tweets when manually inspecting few examples?__ 

__Note: UDPipe approach kept restarting the kernel and would not work for me so I decided to use the nltk.tonkenize.TweetTokenizer approach.__

UsageError: Line magic function `%?` not found.


In [4]:
from nltk.tokenize import TweetTokenizer
tknzr = TweetTokenizer()
result= tknzr.tokenize(data[0])
print(result)
data_tok = [tknzr.tokenize(d) for d in data]

['check', 'out', 'my', 'class', 'in', '#granbluefantasy', '!']


#### Comments on tokenization:
It seems to work quite well. It even tokenizes the emojis. Unfortunately it does not handle contractions. Again, I would have used UDPipe.pipeline if it didn't constantly restart my notebook kernel.

In [5]:
token_counter = Counter()
for tweet in data:
    tokenized = tknzr.tokenize(tweet)
    token_counter.update(tokenized)

__4. Count a word frequency list (how many times each word appears and how many              unique words there are). Which are the most common words appearing in the data?              What kind of words these are?__

In [6]:
nltk.download('stopwords') # download the stopwords dataset

from nltk.corpus import stopwords

filtered_tokens = []

# list of punctuation symbols to ignore
punctuation_chars = '. , : ( ) ! ? " = & - ; ... \\ “ … * ’ / ”' .split() 
punctuation_chars+="'"

for word, count in token_counter.most_common(150):
    if word.lower() in stopwords.words("english") or word in punctuation_chars:
        continue
    filtered_tokens.append((word, count))
print("Number of tokens:", len(filtered_tokens))
print("Tokens:", filtered_tokens)

Number of tokens: 54
Tokens: [('rt', 5801), ('christmas', 517), ('️', 481), ('like', 454), ('😂', 447), ('one', 380), ('love', 361), ('new', 347), ('people', 335), ('get', 309), ('day', 291), ('year', 275), ('2017', 269), ('good', 245), ('time', 241), ('today', 240), ('1', 218), ('see', 216), ('know', 203), ('❤', 202), ('u', 199), ('want', 199), ('best', 193), ('need', 192), ("i'm", 190), ('2018', 190), ('happy', 190), ('got', 186), ('family', 186), ('back', 183), ('make', 178), ('go', 171), ('follow', 170), ('life', 167), ('much', 165), ('thank', 163), ('merry', 161), ('first', 153), ('🎄', 153), ('2', 152), ('great', 152), ('really', 151), ('us', 151), ('..', 150), ('right', 143), ('would', 143), ('😍', 143), ('think', 141), ('even', 141), ('$', 139), ('everyone', 138), ('trump', 136), ('ever', 133), ('😭', 131)]


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\edvar\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


#### Comments on word frequency
It appears that the tweets were collected in the month of december due to the frequent use of 'chrismas','merry','new', 'year' etc. It seems that many of the tweets are retweets, if that's what RT refers to. Which is the case as there are 1270 douplicates in the copus.

As I used 'nlkt.TweetTokenizer' we also have the emojis used. In this case there seem to also be a number of christmas related emojis present.

In [150]:
f'Number of duplicates: {len(data)-len(set(data))}'

'Number of duplicates: 1270'

In [149]:
f'{len(token_counter)} tokens in the data'

'26212 tokens in the data'

__5. Calculate ​idf weight for each word appearing in the data (one tweet = one              document), and print top 20 words with lowest and highest idf values. Why ​tf does               not really matter when processing tweets?__

In [265]:
word_dict = {}
for word, count in filtered_tokens:
    if word.lower() in stopwords.words("english") or word in punctuation_chars:
        continue
    word_dict[word] = 0
    #use the tokenized data in order to accurately count
    # i.e. tokenized tweets 
    for d in data_tok: 
        if word.lower() in d:
            word_dict[word]+=1
            #word_df.update(word)

In [266]:
df_idf = pd.DataFrame()
word_list, idf = [], [] 
for word in word_dict.keys():
    word_list.append(word)
    idf.append(len(data)/word_dict[word])
df_idf['word'] = word_list
df_idf['idf'] = idf
df_idf = df_idf.sort_values('idf')

In [278]:
df_idf.head(20).word.values #20 Lowest IDF

array(['christmas', 'like', 'one', 'love', 'new', 'people', 'get', 'day',
       'year', '2017', 'today', 'good', 'time', 'see', 'know', 'want',
       'need', 'got', 'best', 'happy'], dtype=object)

In [277]:
df_idf.tail(20).word.values #20 Highest idf

array(['layout', 'whyyyyy', '1780', 'byeeeeeeeeeeeeeeeeeeeeee', 'moritz',
       'hatchet', '@shreya1526', 'sidharth', 'malhotra', '@regalglobal',
       'maserati', 'bazalwane', '@gamergod240', 'brockhampton', 'abysmal',
       '@brainpicker', 'scientist', 'coined', 'launched', '@swo22'],
      dtype=object)

### Comments on IDF

It seems that Christmas related words were given the lowest IDF, probably due to their high prevalence during the months close to december which increasingly seems like the time of year these tweets were sampled.

The highest IDFs were observed for pseudowords, words that are not particularly prevalent and some profile taggers.

Term Frequency (TF) would be almost pointless to calculate as tweets are so short.

__6 ​Find duplicate or           near duplicate tweets (in terms of text field only) in the data using any method you                see fit. What kind of techniques you considered using and/or tested, and how many              duplicate or near duplicate did you find?__

In [8]:
# set global parameters to process the whole dataset
bands = 5
rows = 5
nsig = bands*rows  # number of elements in signature, or the number of different random hash functions

maxShingleID = 2**32-1  # record the maximum shingle ID that we assigned
nextPrime = 4294967311  # next prime number after maxShingleID

A = numpy.random.randint(0, nextPrime, size=(nsig,), dtype='int64')
B = numpy.random.randint(0, nextPrime, size=(nsig,), dtype='int64')

In [9]:
def get_shingles(text, k=5):
    """Get all shingles from requested file (hashes of these shingles)
    """
    L = len(text)
    shingles = set()  # we use a set to automatically eliminate duplicates
    for i in range(L-k+1):
        shingle = text[i:i+k]
        crc = binascii.crc32(shingle.encode('utf-8')) #& 0xffffffff  # hash the shingle to a 32-bit integer
        shingles.add(crc)
    return shingles

def jaccard_similarity_score(x, y):
    """
    Jaccard Similarity J (A,B) = | Intersection (A,B) | /
                                    | Union (A,B) |
    """
    intersection_cardinality = len(set(x).intersection(set(y)))
    union_cardinality = len(set(x).union(set(y)))
    return intersection_cardinality / float(union_cardinality)

#jaccard_similarity_score(shingles_vectors[0], shingles_vectors[1])

In [10]:
# fast implementation of Minhash algorithm
# computes all random hash functions for a shingle at once, using vector operations
# also finds element-wise minimum of two vectors efficiently
def minhash_vectorized(shingles, A, B, nextPrime, maxShingleID, nsig):
    signature = numpy.ones((nsig,)) * (maxShingleID + 1)

    for ShingleID in shingles:
        hashCodes = ((A*ShingleID + B) % nextPrime) % maxShingleID
        numpy.minimum(signature, hashCodes, out=signature)

    return signature

In [11]:
def LSH(signatures, bands, rows, Ab, Bb, nextPrime, maxShingleID):
    """Locality Sensitive Hashing
    """
    numItems = signatures.shape[1]
    signBands = numpy.array_split(signatures, bands, axis=0)
    candidates = set()
    for nb in range(bands):
        hashTable = {}
        for ni in range(numItems):
            item = signBands[nb][:,ni]
            hash = (numpy.dot(Ab[nb,:], item) + Bb[nb]) % nextPrime % maxShingleID
            if hash not in hashTable:
                hashTable[hash] = [ni]
            else:
                hashTable[hash].append(ni)
        for _,items in hashTable.items():
            if len(items) > 1:
                L = len(items)
                for i in range(L-1):
                    for j in range(i+1, L):
                        cand = [items[i], items[j]]
                        numpy.sort(cand)
                        candidates.add(tuple(cand))
    return candidates

In [12]:
def get_LSH(fnames, s=.9, k=5):

    # find candidates with LSH

    signatures = []  # signatures for all files
    for fname in fnames:
        shingles = get_shingles(fname, k=k)
        signature = minhash_vectorized(shingles, A, B, nextPrime, maxShingleID, nsig)
        signatures.append(signature)

    # prepare data for LSH
    A2 = numpy.random.randint(0, nextPrime/2, size=(bands, rows), dtype='int64')  # now we need a vector of A parameters for each band
    B2 = numpy.random.randint(0, nextPrime/2, size=(bands, ), dtype='int64')
    signatures = numpy.array(signatures).T  # LSH needs a matrix of signatures, not a list of vectors

    Nfiles = signatures.shape[1]  # number of different files
    t = time()
    candidates = LSH(signatures, bands, rows, A2, B2, nextPrime, maxShingleID)
    t2 = time() - t
    
    return candidates, t2

In [13]:
candidates, _ = get_LSH(data, k=5)

In [14]:
len(candidates)

5721

There are 5721 candidates here because many are retweets of the same tweet. As shown earlier there are 1270 duplicates in the data, probably due to several people retweeting similar tweets and automated messages.

In [15]:
len(data)-len(set(data))

1270

In [93]:
Counter(data).most_common(5)

[('rt @slushiimusic: mic drop @bts_twt ', 30),
 ('your overall health is on your mind today as you anticipate th... more for scorpio ',
  24),
 ('rt @louis_tomlinson: thank you so much for all the birthday messages and i hope everyone had a great christmas ! loads of love',
  22),
 ('i liked a @youtube video ', 22),
 ('an unfinished discussion with a family member may unexpectedly... more for virgo ',
  20)]

We can use set() to remove the duplicates, or use them.. you could simply use Counter() and find all of the duplicates.

The most common tweet is a tweet that has been retweeted 30 times. So many duplicates will consist of several people retweeting the same thing.

Some seem like they are automated, for example the 'i liked a @youtube video', probably linked to their youtube account etc. 

Then there are some that are related to horoscopes (i.e. one references virgo and the other scorpio). Again, automated messages.

Someone might be interested in duplicates, _I have chosen to ignore them so that I can more sanely compare the performance of the Local Sensitivity Hashing that I have deployed here._

In [94]:
data_set = list(set(data))
candidates_set, _ = get_LSH(data_set, k=5)
len(candidates_set)

194

In [108]:
def print_pair(n=42):
    print(f'Candidate pair #{n}:')
    print(data_set[list(candidates_set)[n][0]], data_set[list(candidates_set)[n][1]])
    print()

In [113]:
print_pair(2), print_pair(33) ,print_pair(), print_pair(15)

Candidate pair #2:
#videomtv2017 louis tomlinson 🎂 #videomtv2017 louis tomlinson 👗

Candidate pair #33:
@5hvotingstats b #videomtv2017 fifth harmony @5hvotingstats ff #videomtv2017 fifth harmony

Candidate pair #42:
2 people unfollowed me // automatically checked by  4 people followed me and 4 people unfollowed me // automatically checked by 

Candidate pair #15:
rt @football__tweet: harry kane has now scored as many hat-tricks (6) as every other player in the premier league combined in 2017.

that i… rt @amaradatia: harry kane has now scored as many hat-tricks (6) as every other player in the premier league combined in 2017.

that is a m…



(None, None, None, None)

194 candidate pairs were flagged.

As we can see, the algorithm identified similar hashtags with different emojis. The '#videomtv2017 louis tomlinson' is extremely prevalent in this dataset. It seems that this person had their birthday in that month and is/was a popular person on twitter.

It sill identified automated messages, for example '_N_ number of people __unfollowed me__' and '_N_ number of people __followed me__', which are similar.

Candidate pair 15 found a tweet with a retweet and a subsequent comment.

I chose to use LSH hashing with Jaccard Similarity (I could have also used Cosine Distance) as it is fairly simple and fast. I could have also used minHashing or pairwise comparison. I ruled out pairwaise comparison simply because it is so slow, but it would have reduced false positives and false negatives.

##### Summation

One could filter through the data and ignore retweets as they will obviously be flagged by LSH, but it really depends on what one is trying to do. This method seems to work fairly well and is relatively fast.

One could also choose to ignore the '#....' and/or '@....' in order to  analyse the text content only. Otherwise, the hashtags and tags can and do influence this pipeline. Again, it really depends on what the purpose is. I have included them here to keep things simple.

You could also filter out the automated messages if you had a list of automated messages that are generated in twitter.
